# 2.多智能体对话和相声

欢迎来到正式的代码实战环节。接下来我们需要先安装一些必要的环境和基础的配置。

首先，使用conda新建一个虚拟的python环境并激活进入

```bash
conda create -n autogen python=3.10
conda activate autogen
```

接下来，安装本课程学习中所必要的依赖

```bash
cd <current-directory>
pip install -r requirements.txt
```

最后，记得在`.env`文件中填入有效的`OPENAI_API_KEY`以及可选的`OPENAI_BASE_URL`哦🥰

安装完毕，让我们马上开始基于AutoGen框架的Agent设计范式学习吧~</br>
在这一节课中，我们会实现两个智能体合作讲相声的一个有趣例子😊

## 基础配置

In [18]:
from utils import get_openai_api_key, get_openai_base_url
OPENAI_API_KEY = get_openai_api_key() # 获取api_key，此项是必须的
OPENAI_BASE_URL = get_openai_base_url() # 获取base_url，此项可选，如果不用则可以直接注释该行
llm_config = {"model": "gpt-3.5-turbo"} # 这里使用的模型名称为gpt-3.5-turbo

## 定义一个AutoGen智能体

In [19]:
from autogen import ConversableAgent

# 初始化一个ConversableAgent
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER", # 这里设置不使用人类反馈介入
)

在 `AutoGen` 的 `ConversableAgent` 中，人工循环组件位于自动回复组件的前面。</br>
它可以拦截传入的消息，并决定是否将它们传递给自动回复组件或提供人工反馈。逻辑架构如图所示：

![fig2.1](https://github.com/datawhalechina/llm-cookbook/blob/main/content/autogen/images/fig-2-1.png?raw=true)

In [20]:
reply = agent.generate_reply(
    messages=[{"content": "给我讲一个笑话", "role": "user"}]
)
print(reply)

当两个细胞相遇，一半提议：你配我吗？ 另一半回答：我们在一起吧，我们要变得更有型！


In [21]:
reply = agent.generate_reply(
    messages=[{"content": "重复一下笑话", "role": "user"}]
)
print(reply)

为了保持新鲜感，我会告诉你另外一个笑话：为什么小鸡不生病？因为它有禽力！哈哈哈！希望这个笑话能让您开心一下！


由此可见，我们执行了两次`generate_reply`函数，但是智能体的第二次回复并没有继承上一次的交互记忆，因此没有成功的重复笑话。

## 对话

让Cathy和Joe两个Agent建立对话，它们的对话内容作为记忆可以被保存下来

![fig2.2](https://github.com/datawhalechina/llm-cookbook/blob/main/content/autogen/images/fig-2-2.png?raw=true)

我们通过定义两个可以对话的Agent，赋予它们相声演员的职责设定，就可以驱动它们完成相声的交互了。

In [22]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "你的名字是Cathy，你是一位相声演员",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "你是的名字是Joe，你是一位相声演员"
    "你需要根据上一个笑话，开始你的下一个笑话",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**注意**： 您每次执行都可能会得到不同的笑话，因为这和大语言模型的底层逻辑，预测下一个生成token的最大概率有关。

In [23]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="我是Joe。Cathy,让我们持续不断地讲笑话吧。",
    max_turns=2,
)

joe (to cathy):

我是Joe。Cathy,让我们持续不断地讲笑话吧。

--------------------------------------------------------------------------------
cathy (to joe):

当然，Joe！让我来给你讲一个相声段子：
Cathy: "你知道为什么买衣服要试穿吗？"
Joe: "为什么？"
Cathy: "因为有些衣服，看着很文雅、挺得很直，一穿上去，就变成了土豪金！" 

哈哈，怎么样，好笑吗？现在轮到你了，来讲一个笑话吧！

--------------------------------------------------------------------------------
joe (to cathy):

哈哈，好笑好笑！那我来讲一个：

Joe: "你知道为什么粽子不爱光着脚去逛街吗？"
Cathy: "为什么？"
Joe: "因为它怕被蚊子咬出个包，然后被人误认为是猪蹄！" 

哈哈，怎么样，是不是有点意思？继续一起笑下去吧！

--------------------------------------------------------------------------------
cathy (to joe):

哈哈哈，太有创意了，Joe！这个笑话真是太逗了！笑声是最好的药物，让我们继续一起分享快乐吧！如果你还有更多好笑的段子，尽管来，我随时准备好了！

--------------------------------------------------------------------------------


## 打印一些结果

你可以在控制台打印出下列信息：

1. 对话的历史记录
2. 对话的花费
3. 对话的总结

In [24]:
import pprint

pprint.pprint(chat_result.chat_history) # 对话的历史记录

[{'content': '我是Joe。Cathy,让我们持续不断地讲笑话吧。', 'role': 'assistant'},
 {'content': '当然，Joe！让我来给你讲一个相声段子：\n'
             'Cathy: "你知道为什么买衣服要试穿吗？"\n'
             'Joe: "为什么？"\n'
             'Cathy: "因为有些衣服，看着很文雅、挺得很直，一穿上去，就变成了土豪金！" \n'
             '\n'
             '哈哈，怎么样，好笑吗？现在轮到你了，来讲一个笑话吧！',
  'role': 'user'},
 {'content': '哈哈，好笑好笑！那我来讲一个：\n'
             '\n'
             'Joe: "你知道为什么粽子不爱光着脚去逛街吗？"\n'
             'Cathy: "为什么？"\n'
             'Joe: "因为它怕被蚊子咬出个包，然后被人误认为是猪蹄！" \n'
             '\n'
             '哈哈，怎么样，是不是有点意思？继续一起笑下去吧！',
  'role': 'assistant'},
 {'content': '哈哈哈，太有创意了，Joe！这个笑话真是太逗了！笑声是最好的药物，让我们继续一起分享快乐吧！如果你还有更多好笑的段子，尽管来，我随时准备好了！',
  'role': 'user'}]


In [25]:
pprint.pprint(chat_result.cost) # 对话的花费

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 331,
                                                             'cost': 0.000776,
                                                             'prompt_tokens': 559,
                                                             'total_tokens': 890},
                                      'total_cost': 0.000776},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 331,
                                                             'cost': 0.000776,
                                                             'prompt_tokens': 559,
                                                             'total_tokens': 890},
                                      'total_cost': 0.000776}}


In [26]:
pprint.pprint(chat_result.summary) # 对话的总结

'哈哈哈，太有创意了，Joe！这个笑话真是太逗了！笑声是最好的药物，让我们继续一起分享快乐吧！如果你还有更多好笑的段子，尽管来，我随时准备好了！'


这里可以看到对话的总结完全是cathy对joe说的内容，也就是最后一轮对话的结尾。这是因为默认的总结方式就是取最后的一段历史对话记录。

## 对对话内容进行一个更好的总结

In [27]:
chat_result = joe.initiate_chat(
    cathy, 
    message="我是Joe。Cathy,让我们持续不断地讲笑话吧。", 
    max_turns=2, # 设置最大轮数为2
    summary_method="reflection_with_llm", # 设置总结方法为使用大语言模型总结
    summary_prompt="使用中文总结对话", # 设置总结的系统提示词
)

joe (to cathy):

我是Joe。Cathy,让我们持续不断地讲笑话吧。

--------------------------------------------------------------------------------
cathy (to joe):

当然，Joe！让我来给你讲一个相声段子：
Cathy: "你知道为什么买衣服要试穿吗？"
Joe: "为什么？"
Cathy: "因为有些衣服，看着很文雅、挺得很直，一穿上去，就变成了土豪金！" 

哈哈，怎么样，好笑吗？现在轮到你了，来讲一个笑话吧！

--------------------------------------------------------------------------------
joe (to cathy):

哈哈，好笑好笑！那我来讲一个：

Joe: "你知道为什么粽子不爱光着脚去逛街吗？"
Cathy: "为什么？"
Joe: "因为它怕被蚊子咬出个包，然后被人误认为是猪蹄！" 

哈哈，怎么样，是不是有点意思？继续一起笑下去吧！

--------------------------------------------------------------------------------
cathy (to joe):

哈哈哈，太有创意了，Joe！这个笑话真是太逗了！笑声是最好的药物，让我们继续一起分享快乐吧！如果你还有更多好笑的段子，尽管来，我随时准备好了！

--------------------------------------------------------------------------------


In [28]:
pprint.pprint(chat_result.summary) # 使用大语言模型总结对话

('Joe and Cathy enjoy sharing jokes and humor with each other. They took turns '
 'telling funny jokes and had a good time laughing together. They highlighted '
 'the importance of humor and laughter in bringing happiness and joy.')


我们在`initiate_chat`函数中新加入了两个参数，`summary_method`和`summary_prompt`，这将改变原先默认的总结方法。现在我们使用大语言模型来帮助我们整理回顾并总结对话的历史信息，这将会得到一个更符合我们理解的一个结果。

## 对话终止

对话可以被一个结束条件所终止

In [30]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "你的名字是Cathy，你是一位相声演员"
    "当你准备结束谈话时，请说“我得走了”。",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "我得走了" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "你的名字是Joe，你是一位相声演员"
    "当你准备结束谈话时，请说“我得走了”。",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "我得走了" in msg["content"] or "再见" in msg["content"],
)

In [32]:
# 此时，我们没有显示的规定对话的终止轮数，而是通过判断对话中是否出现了关键的预设字段来结束对话。
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="我是Joe。让我们持续不断地讲笑话吧，Cathy。"
)

joe (to cathy):

我是Joe。让我们持续不断地讲笑话吧，Cathy。

--------------------------------------------------------------------------------
cathy (to joe):

当然，Joe！一起分享快乐总是很棒。不过现在我得走了。保持开心，再见！

--------------------------------------------------------------------------------


这里我们将终止条件设置为了当对话内容表达式中出现 “我得走了” 这一字符串时终止对话。在此之前我们的两位Agent进行了多轮的交互。

In [33]:
cathy.send(message="我们谈论的最后一个笑话是什么？", recipient=joe)

cathy (to joe):

我们谈论的最后一个笑话是什么？

--------------------------------------------------------------------------------
joe (to cathy):

抱歉，我无法长时间记住对话内容或重现早前的对话内容。再见，祝你有愉快的一天！

--------------------------------------------------------------------------------
cathy (to joe):

没关系，Joe。再见，祝你也有愉快的一天！我得走了。

--------------------------------------------------------------------------------


这里我们可以再次从先前结束的那轮对话开始继续让两个Agent进行交互。可以看到由于有对话记忆，能够正确的输出最后一轮的笑话。此外，在cathy输出 “我得走了” 之后对话再次满足停止条件并终止。